# This notebook was originally ran on Google Colab, local running will need modification

In [ ]:
# === Imports === #

# My imports
import os
import subprocess
import pandas as pd
from tqdm import tqdm
import shutil

# Various python packages are used in this notebook. Please get yourself used to them (optional).
import pandas as pd  # used for storing a tabular representation of the dataset, similar to XLS files.
from pathlib import Path # used to check if the saved model files and accessories.
import requests #used to request remote judge.csv evaluation 
from sklearn.preprocessing import StandardScaler  # used for normalization of dataset
from sklearn.preprocessing   import LabelBinarizer    # used for splitting the gender column
from sklearn.preprocessing   import MinMaxScaler      # used for normalization of dataset
from sklearn.model_selection import train_test_split  # used for performing the train-test split of a dataframe
import cv2                                            # OpenCV used for image processing
import random   #random number generator
import datetime #used to get current date/time
import math     #math/numerical functions
import os       #os specific functions, like file open/close etc.
import gc       #garbage collection module -- used to manually clean up memory spaces/references.


from sklearn.preprocessing import OneHotEncoder   #My favorite categorical to numerical feature conversion tool
from tensorflow import keras  # keras used for construction of the Artificial neural network
from keras.models import Sequential #keras model architectures
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten,Dropout, BatchNormalization #types of layers
from keras.losses import mean_squared_error, huber, log_cosh  #built-in loss 
from tensorflow.python.keras.saving import hdf5_format  #used for saving models 
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard  #callbacks
from keras.models import model_from_json  #used for loading model architecture from json file
import h5py  #saved model type

import matplotlib.pyplot as plt  # used for training visualization
import numpy as np  # numpy arrays used for matrix computations

# === Extra Configurations for the GPU Environment === #
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0: #If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
"""
Setting work environment with dataset. If on Google colaboratory, we need to extract dataset stored in google drive,
otherwise the dataset is already there.
"""
try:
    from google.colab import drive
    print('Running on Google colab...')
    drive.mount('/content/drive')
    #import zipfile
    #with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/csci5931sp22/PA2-dataset/Archive.zip', 'r') as zip_ref:
    #    zip_ref.extractall('dataset/')
except:
    print('Running on local machine...')

Running on Google colab...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = './drive/MyDrive/Colab Notebooks/DAiSEE/'

#dataset = os.listdir('DataSet')
all_labels = pd.read_csv(os.path.join(folder_path, 'Labels/AllLabels.csv'))

file_extension = '.avi'
sample_num = '5000441001' + file_extension

all_labels[all_labels['ClipID']==sample_num].values.tolist()[0][-4:]

[1, 2, 0, 0]

In [ ]:
subset_directory = os.path.join(folder_path, 'DataSet')
subsets = ['Test', 'Train', 'Validation']

generated_folder = 'gen-full-4'
resize_scale = "299:224"

if generated_folder in os.listdir(folder_path):
    shutil.rmtree(os.path.join(folder_path, generated_folder))
os.makedirs(os.path.join(folder_path, generated_folder))
target_directory = os.path.join(folder_path, generated_folder)

for subset in subsets:
    subset_path = os.path.join(subset_directory, subset)

    os.makedirs(os.path.join(folder_path, generated_folder, subset))

    persons = os.listdir(subset_path)
    for person in tqdm(persons):
        person_path = os.path.join(subset_path, person)
        person_videos = os.listdir(person_path)
        for person_video in person_videos:
            try:

                person_video_path = os.path.join(person_path, person_video)
                
                if len(os.listdir(person_video_path)) == 0:
                    continue

                filename = os.listdir(person_video_path)[0]
                file_path = os.path.join(person_video_path, filename)
                
                new_filename = person_video + "-%d.jpg"

                target_path = os.path.join(target_directory, subset, new_filename)

                '''
                Gen2 = 2 hz = 0.5 seconds between frames
                Gen3 = .5 hz = 2 seconds between frames
                '''

                command = 'ffmpeg -i "' + file_path + '" -vf scale=' + resize_scale + ' -r 0.5 -qscale:v 2 "' + target_path + '"'
                os.system(command)
            except:
                print("error with " + person_video)


100%|██████████| 22/22 [40:24<00:00, 110.19s/it]


In [ ]:
print("Success...")

Success...


https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3